In [ ]:
#| default_exp to_emb_atlas

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

In [ ]:
#| export

import atexit
import zipfile
from pathlib import Path
from tempfile import TemporaryDirectory as TmpDir

import daft
import embedding_atlas.cli as emb_atlas_cli
import polars as pl
from daft.functions import to_struct

from clip_plot.utils import timestamp


#

1. concatenate all data into a dataframe
2. load images as base strings
3. run atlas creation
4. unzip upon exit

- path is relative to the output folder; can turn into a link with <a>
- need to check out huggingface example to see why image renders properly
- should we drop _row_index?
- embed_path is there in test, but it's just because it's in the input, nbd

In [ ]:
#| export

@daft.func
def relative_image_path(img_path: str, viewer_dir: Path) -> str:
    return str(Path(img_path).relative_to(viewer_dir))

In [ ]:
#| export

def load_images(df: daft.DataFrame, image_path_col: str,
                viewer_dir: Path) -> daft.DataFrame:
    df = (df
    .with_column("image_bytes", daft.col(image_path_col).url.download(on_error="null"))
    .with_column("image", daft.col("image_bytes").image.decode())
    .with_column("image_jpeg_bytes", daft.col("image").image.encode("JPEG"))
    # .with_column("image_jpeg_base64", daft.col("image_jpeg_bytes").encode("base64").decode("utf-8"))
    # .with_column("image_relpath", relative_image_path(df[image_path_col], viewer_dir))
    # .with_column("image", to_struct(daft.col("image_jpeg_bytes"), daft.col(image_path_col)))
    )
    # df = df.with_column("image_data_url",
    #                     daft.functions.concat(daft.lit("data:image/jpeg;base64,"),
    #                     df["image_jpeg_base64"]))
    df = df.with_column("image", to_struct(bytes=df["image_jpeg_bytes"], path=df[image_path_col])
    ).exclude("image_bytes", "image_jpeg_bytes")

    front_cols = ["image", image_path_col]
    # drop_cols = ["image_bytes", "image_jpeg_bytes", "image_jpeg_base64"]
    others = sorted(c for c in df.column_names if c not in front_cols)
    keep = front_cols + others
    print(f"Columns to keep: {keep}")
    return df.select(*keep)

In [ ]:
#| export

def unzip_atlas(zip_path: Path, extract_target: Path, delete_after: bool = False):
    """unzip at exit"""
    print(timestamp(), f"Extracting the viewer bundle to \n{extract_target.as_posix()}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_target)
    if delete_after:
        zip_path.unlink()
    return None

In [ ]:
#| export

def run_emb_atlas(parquet_path: Path, zip_path: Path,
                  viewer_dir: Path, temp_dir: TmpDir):
    atexit.register(print,
                    f"{timestamp()} Finished creating atlas viewer at {viewer_dir}")
    atexit.register(temp_dir.cleanup)
    # last registered runs first
    atexit.register(unzip_atlas,
                    zip_path=zip_path, extract_target=viewer_dir
                    )
    emb_atlas_cli.main.main(
        args=[parquet_path.as_posix(),
            "--x", "emb_x", "--y", "emb_y",
            "--export-application", zip_path.as_posix(),
            ],
        standalone_mode=False
        )
    return None

In [ ]:
#| export

def create_emb_atlas(table: pl.DataFrame, image_path_col: str,
                     viewer_dir: Path, plot_id: str) -> None:
    df = daft.from_arrow(table.to_arrow())
    df = load_images(df, image_path_col, viewer_dir)
    prep_dir = TmpDir()
    parquet_path = Path(prep_dir.name) / f"viewer-input-{plot_id}.parquet"
    zip_path = parquet_path.with_suffix(".zip")
    df.write_parquet(parquet_path, write_mode="overwrite")
    run_emb_atlas(parquet_path, zip_path=zip_path,
                    viewer_dir=viewer_dir, temp_dir=prep_dir)
    return None

NameError: name 'pl' is not defined

In [ ]:
#| hide

import nbdev

nbdev.nbdev_export()